# Forecasting

## Loading Libraries and datasets

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
#from matplotlib import pyplot as plt

In [5]:
# importing pyplot from matplotlib as plt
# for plotting graphs
import matplotlib.pyplot as plt
plt.style.use('default')
%matplotlib inline

In [3]:
df_train = pd.read_csv('../competitive-data-science-final-project/new_sales_train.csv')

In [4]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_cnt_day,Month,Year,DayofWeek,item_category_id,Holiday
0,2013-01-01,0,2,991,1.0,1,2013,1,67,True
1,2013-01-01,0,2,1472,1.0,1,2013,1,23,True
2,2013-01-01,0,2,1905,1.0,1,2013,1,30,True
3,2013-01-01,0,2,2920,2.0,1,2013,1,21,True
4,2013-01-01,0,2,3320,1.0,1,2013,1,19,True


In [8]:
df_train.groupby(["date", "shop_id", "item_id"])[["item_cnt_day"]].count().item_cnt_day.value_counts()

1    2882308
Name: item_cnt_day, dtype: int64

In [ ]:
#Each row is unique for date, shop_id, item_id combination

## 1. Moving average